<a href="https://colab.research.google.com/github/joeban0608/python_liontravel_project/blob/main/liontravel_keyword_worldcould_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# get_godatastart & godataend dates 
# 想要出發的行程區間,預設[當日]至[當日再往後加30日]
# 參考來源:https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/369869/
import datetime
import time
from datetime import timedelta, date
 
# 取[今日的str日期]方式
def get_liontravel_datas():
    def strTodatetime(datestr, format):
        return datetime.datetime.strptime(datestr, format)
    data_godatestart = time.strftime("%Y-%m-%d", time.localtime())
 
    # 取[未來30天的日期]方式
    def get_day_of_day(n=0):
        if(n<0):
            n = abs(n)
            return date.today()-timedelta(days=n)
        else:
            return date.today()+timedelta(days=n)
    data_godateend = str(get_day_of_day(30))
    return data_godatestart , data_godateend

In [ ]:
# get_fulltext
# 下載jieba 大字典
import jieba
from urllib.request import urlretrieve
import requests
from bs4 import BeautifulSoup
 
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.big") 
 
jieba.set_dictionary("dict.big")  # 設定好原始字典
# 看情況加入自定義字典
jieba.load_userdict("liontravel_title.tit") # 載入自定義字典
 
data_godatestart = get_liontravel_datas()[0]
data_godateend  = get_liontravel_datas()[1]
 
def get_liontravel_fulltext(input_SortType=1, input_Keywords=4,data_godatestart=data_godatestart,data_godateend=data_godateend):
    # 由於畫出文字雲需要把所有文字整理成一篇文章, 於是我們準備一個容器來儲存
    fulltext = ""
    # 給網頁search的參數, 預設值台灣，依價格搜尋
    list_KEYWORDS = ["一日遊","台東","宜蘭","基隆","台灣"]
    PAGE = 1
    i = 0
    while True:
        url = "https://travel.liontravel.com/search/grouplist"
        GODATESTART = data_godatestart
        GODATEEND = data_godateend
        SORTYPE = str(input_SortType)
        KEYWORDS = list_KEYWORDS[input_Keywords]
        datas = {
                "ArriveID":",",
                "GoDateStart": GODATESTART,
                "GoDateEnd": GODATEEND,
                "Keywords": KEYWORDS,
                "IsEnsureGroup": "false",
                "IsSold": "true",
                "GoStartDays": "0",
                "TravelType": "1",
                "TourArea": "0",
                "Page": str(PAGE + i),
                "SortType": SORTYPE
                }
        if not datas["Page"] == "1":
           url = url + "item" 
        response = requests.post(url, datas)
        html = BeautifulSoup(response.text)
        travels = html.find_all("div",class_="rli_tlin")
        # if not 群集: 參考用法by東哲
        if not travels:
            break
        for travel in travels:
            # 將往返放回全文，因為後面jieba分析關鍵詞有用到，觀念提醒by家和
            title = travel.find("div",class_="rli_body").find("div",class_="rli_content").find("a")
            title = title.text.replace("⇄","往返")
            fulltext = fulltext + " " + " ".join(jieba.cut(title))
        i += 1     
    return fulltext, list_KEYWORDS[input_Keywords]

In [ ]:
# get_fulltext, filename
# 運用寫好的函式，調整input_Keywords=參數，拿到想要的旅遊篇幅
fulltext = get_liontravel_fulltext(input_Keywords=0)[0]
filename = get_liontravel_fulltext(input_Keywords=0)[1]
# print(fulltext)
# input_keywords=0 一日遊
# input_keywords=1 台東
# input_keywords=2 宜蘭
# input_keywords=3 基隆
# input_keywords=4 [預設台灣]

In [ ]:
# get_keyword
# extract_tags() 抓出10個跟搜尋關鍵字有關的熱門關鍵字
# 運用jieba.del_word() 刪掉不重要的關鍵詞 
from jieba.analyse import extract_tags
def get_keyword(top=10):
  jieba.del_word("出發")
  list_keyword = extract_tags(fulltext, top)
  # 因為文字雲，會將放在越前面的list號碼的文字加大，因此將重要的關鍵詞改放排序方式:
  # 一日遊篇,發現一日遊被排在第四個,藉由list method找出一日遊刪掉,並將一日遊放回第一個
  if not list_keyword[0] == filename:
    for i in list_keyword:
      if i == filename:
        list_keyword.remove(i)
        list_keyword.insert(0, filename)
  # 將list_keyword轉換方式，參考by家和
  keyword = str(list_keyword)[1:-2].replace(",", "").replace("'", "")
  return keyword
# top可調整要取幾個關鍵詞
keyword = get_keyword(top=10)
# print(keyword)

In [ ]:
# 除了安裝wordcloud 另外記得把1.文字雲的背景圖片"jpg檔"& 2.google字體的檔案丟到ipynb資料夾 
# 參考by東哲
!pip install wordcloud

In [ ]:
# plot wordcloud
# 參考by又益，周老師
from PIL import Image
import numpy as np
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
 
# 先把我們要做文字雲的圖片開起來
mask_path = "TaiwanMap.jpg"
mask = np.array(Image.open(mask_path))
# 準備WordCloud
wc = WordCloud(font_path="./NotoSansCJKtc-Bold.otf", 
               background_color="white", 
               max_words=5000,
               mask=mask,
               collocations=False)
wc.generate(keyword)
# 把原圖的色彩擷取出來, 而且用來上色wc(文字雲)
color = ImageColorGenerator(mask)
wc.recolor(color_func = color)
# 儲存檔案, 你可在你的project 看到這個檔案
wc.to_file("./" + filename + "_cloud.png")
# (Jupyter Only) 你可以在Notebook 上直接秀出
wc.to_image()